In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import requests
import sqlite3
from typing import List, Union
from datetime import datetime, timedelta
import numpy as np


class XMLDataLoader:
    """
    Base class for loading and processing XML data from a list of URLs.
    """
    def __init__(self, urls: List[str], db_name: str, table_name: str):
        """
        Initialize XMLDataLoader with a list of URLs and a database name.

        :param urls: List of URLs to fetch XML data from.
        :param db_name: Name of SQLite database to store data.
        :param table_name: Name of the table in the SQLite database.
        """
        self.urls = urls
        self.db_name = db_name
        self.table_name = table_name
        self.data = []

    def fetch_and_parse_data(self) -> 'XMLDataLoader':
        """
        Fetch XML data from URLs and parse it.

        This method should be overridden in subclasses to provide the specific parsing logic for each type of report.

        :return: self
        """
        pass

    def to_dataframe(self) -> 'XMLDataLoader':
        """
        Convert fetched data to a Pandas DataFrame.

        :return: self
        """
        self.df = pd.DataFrame(self.data)
        return self

    def to_sql(self) -> None:
        """
        Write DataFrame to SQLite database.

        :return: None
        """
        with sqlite3.connect(self.db_name) as conn:
            self.df.to_sql(self.table_name, conn, if_exists='replace', index=False)


class DailyLoadForecastDataLoader(XMLDataLoader):
    """
    Class for loading and processing Daily Load Forecast XML data from a list of URLs.
    """
    def fetch_and_parse_data(self) -> 'DailyLoadForecastDataLoader':
        """
        Fetch XML data from URLs and parse it.

        :return: self
        """
        for url in self.urls:
            response = requests.get(url)
            root = ET.fromstring(response.content)

            for pub_daily_load in root.findall('PUB_DailyLoadFcst'):
                row_data = pub_daily_load.attrib
                row_data.update(root.attrib)

                for element in pub_daily_load:
                    row_data[element.tag] = element.text

                self.data.append(row_data)

        return self


class FourDayAggForecastDataLoader(XMLDataLoader):
    """
    Class for loading and processing Four Day Aggregate Forecast XML data from a list of URLs.
    """
    def fetch_and_parse_data(self) -> 'FourDayAggForecastDataLoader':
        """
        Fetch XML data from URLs and parse it.

        :return: self
        """
        for url in self.urls:
            response = requests.get(url)
            root = ET.fromstring(response.content)

            for element in root.findall('PUB_4DayAggRollWindUnitFcst'):
                row_data = element.attrib
                row_data.update(root.attrib)
                self.data.append(row_data)

        return self



class BidAskCurvesDataLoader(XMLDataLoader):
    """
    Class for loading and processing BidAskCurves XML data from a list of URLs.
    """
    def fetch_and_parse_data(self):
        """
        Fetch XML data from URLs and parse it.

        This method parses BidAskCurves XML data.
        """
        for url in self.urls:
            response = requests.get(url)
            root = ET.fromstring(response.content)

            for MarketArea in root.iter("MarketArea"):
                MarketAreaName = MarketArea.find("MarketAreaName").text
                DeliveryDay = MarketArea.find("DeliveryDay/Day").text

                for t_step in MarketArea.iter("TimeStep"):
                    TimeStep = t_step.find("TimeStepID").text

                    for purchase in t_step.iter("Purchase"):
                        Price = purchase.find("Price").text
                        Volume = purchase.find("Volume").text

                        self.data.append({
                            "MarketArea": MarketAreaName,
                            "DeliveryDay": DeliveryDay,
                            "TimeStep": TimeStep,
                            "Price": Price,
                            "Volume": Volume,
                            "Type": "Purchase"
                        })

                    for sell in t_step.iter("Sell"):
                        Price = sell.find("Price").text
                        Volume = sell.find("Volume").text

                        self.data.append({
                            "MarketArea": MarketAreaName,
                            "DeliveryDay": DeliveryDay,
                            "TimeStep": TimeStep,
                            "Price": Price,
                            "Volume": Volume,
                            "Type": "Sell"
                        })
        return self



class WeatherData(XMLDataLoader):
    """
    Class for loading and processing Weather data from a weather API.
    """
    def __init__(self, latitude: float, longitude: float, db_name: str, table_name: str):
        """
        Initialize WeatherData with a latitude and longitude.

        :param latitude: Latitude of the location.
        :param longitude: Longitude of the location.
        """
        super().__init__([], db_name, table_name)
        self.latitude = latitude
        self.longitude = longitude
        self.parameters = []
        self.url_base = "https://api.open-meteo.com/v1/forecast"

    def add_parameters(self, params: List[str]) -> 'WeatherData':
        """
        Add a list of parameters to the WeatherData instance.

        :param params: List of parameters to be fetched from the API.
        """
        self.parameters.extend(params)
        return self

    def set_date_range(self, start_date: str, end_date: str) -> 'WeatherData':
        """
        Set the date range for the weather data.

        :param start_date: Start date in 'YYYY-MM-DD' format.
        :param end_date: End date in 'YYYY-MM-DD' format.
        """
        self.start_date = start_date
        self.end_date = end_date
        return self

    def fetch_and_parse_data(self) -> Union['WeatherData', None]:
        """
        Fetch weather data from the API.
        """
        url = f"{self.url_base}?latitude={self.latitude}&longitude={self.longitude}&hourly={','.join(self.parameters)}&start_date={self.start_date}&end_date={self.end_date}"
        try:
            response = requests.get(url)
            response.raise_for_status()
            weather_data = response.json()
            self.data = weather_data['hourly']
            return self
        except requests.HTTPError as e:
            print(f"Request failed with status code {response.status_code}. Error: {str(e)}")
            return None



# Usage
daily_load_forecast_urls = [
    "https://reports.sem-o.com/documents/PUB_DailyLoadFcst_202306040650.xml",
    # Add more URLs as needed
]
four_day_agg_forecast_urls = [
    "https://reports.sem-o.com/documents/PUB_4DayAggRollWindUnitFcst_202306050011.xml",
    # Add more URLs as needed
]

bid_ask_curves_urls = [
    "https://reports.semopx.com/documents/BidAskCurves_SEM-IDA3_20220610_20220610151630.xml",
    # Add more URLs as needed
]




daily_loader = DailyLoadForecastDataLoader(daily_load_forecast_urls, 'semo.db', 'daily_load_forecast')
four_day_loader = FourDayAggForecastDataLoader(four_day_agg_forecast_urls, 'semo.db', 'four_day_agg_forecast')
bid_ask_loader = BidAskCurvesDataLoader(bid_ask_curves_urls, 'semo.db', 'bid_ask_curves')



daily_loader.fetch_and_parse_data().to_dataframe().to_sql()
four_day_loader.fetch_and_parse_data().to_dataframe().to_sql()
bid_ask_loader.fetch_and_parse_data().to_dataframe().to_sql()



# Weather API parameters
params = [
     'temperature_2m',
    'relativehumidity_2m',
    'dewpoint_2m',
    'apparent_temperature',
    'precipitation_probability',
    'precipitation',
    'rain',
    'showers',
    'snowfall',
    'snow_depth',
    'weathercode',
    'pressure_msl',
    'surface_pressure',
    'cloudcover',
    'visibility',
    'windspeed_10m',
    'windspeed_80m',
    'windspeed_120m',
    'windspeed_180m',
    'winddirection_10m',
    'winddirection_80m',
    'winddirection_120m',
    'winddirection_180m',
    'windgusts_10m',
    'temperature_80m',
    'temperature_120m',
    'temperature_180m'
]

# Generate the start and end dates dynamically
start_date = datetime.utcnow().strftime("%Y-%m-%d")
end_date = (datetime.utcnow() + timedelta(days=7)).strftime("%Y-%m-%d")

# Define locations
locations = {
    'Dublin': (53.3498, -6.2603),
    'Cork': (51.8985, -8.4756),
    'Belfast': (54.5973, -5.9301),
    'Galway': (53.2707, -9.0568)
}

# Fetch weather data for each city and store in the database
for city, (latitude, longitude) in locations.items():
    weather_data = (
        WeatherData(latitude=latitude, longitude=longitude, db_name='weather.db', table_name=city.lower())
        .add_parameters(params)
        .set_date_range(start_date=start_date, end_date=end_date)
        .fetch_and_parse_data()
    )
    if weather_data is not None:
        weather_data.to_dataframe().to_sql()


In [ ]:
import sqlite3
import pandas as pd

# Create a connection to the SQLite database
conn = sqlite3.connect('semo.db')

# Create a DataFrame from a SQL query
df = pd.read_sql_query("SELECT * FROM bid_ask_curves", conn)

# Close the connection
conn.close()

# Display the DataFrame
df.head()

,MarketArea,DeliveryDay,TimeStep,Price,Volume,Type
0,SEM-IDA3,10/06/2022,18HH1,-150,729.9,Purchase
1,SEM-IDA3,10/06/2022,18HH1,-120,729.9,Purchase
2,SEM-IDA3,10/06/2022,18HH1,-120,709.9,Purchase
3,SEM-IDA3,10/06/2022,18HH1,-100,709.9,Purchase
4,SEM-IDA3,10/06/2022,18HH1,-100,564.3,Purchase
